In [1]:
import pandas as pd

DATA_PATH = '../data/'
filename  = 'estrepublicain_annee_1999.csv'

df = pd.read_csv(DATA_PATH + filename)


In [2]:
df
print("= dimensions")
print(df.shape)
print("= 5 premieres rangés")
print(df.head())
print("= Noms des colonnes")
print(df.columns)


= dimensions
(30241, 1)
= 5 premieres rangés
                                                text
0  André Bauer, le Bonhomme de St-Dié ; Alain Dag...
1  Le Smash Entente Club de Lunéville (SECL) a re...
2  En tout cas, du côté du PS Dole qui reste en c...
3  « Le nombre des donneurs était en légère baiss...
4  ELOYES._ Les Ramoncenais n'auront tenu qu'une ...
= Noms des colonnes
Index(['text'], dtype='object')


In [3]:
# Changer le nom de la colonne
df.columns = ['paragraphe']
df.columns

Index(['paragraphe'], dtype='object')

In [4]:
# puis renommer colonne en 'text'

df.columns = ['text']
df.columns

Index(['text'], dtype='object')

In [5]:
# Ne prendre que les 1000 premieres rangées

print("avant: {}".format(df.shape))
df = df[0:1000]

print("apres: {}".format(df.shape))



avant: (30241, 1)
apres: (1000, 1)


In [7]:
# Tokenization
from nltk import word_tokenize

# La phrase
sentence = "les médiateurs confirment « l'absence de solution parfaite ». "

tokens = word_tokenize(sentence)

print(tokens)
tokens_on_space = sentence.split(' ')
print(tokens_on_space)


['les', 'médiateurs', 'confirment', '«', "l'absence", 'de', 'solution', 'parfaite', '»', '.']
['les', 'médiateurs', 'confirment', '«', "l'absence", 'de', 'solution', 'parfaite', '».', '']


In [17]:
# ponctuation

import string

print("Liste original de signe de ponctuations")
print("\t{}".format(string.punctuation))

sentence = "les médiateurs confirment « l'absence de solution parfaite ». "
punctuation_chars = string.punctuation + "«»"

#  Construction d'un dict { '~':' ', '$': ' ', ... }
dict_ponctuation = {}
for k in punctuation_chars:
    dict_ponctuation[k] = ' '

print(dict_ponctuation)

# # L'operateur de translation
translator = str.maketrans(dict_ponctuation)

new_sentence = sentence.translate(translator)
print(sentence)
print(new_sentence)


Liste original de signe de ponctuations
	!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
{'!': ' ', '"': ' ', '#': ' ', '$': ' ', '%': ' ', '&': ' ', "'": ' ', '(': ' ', ')': ' ', '*': ' ', '+': ' ', ',': ' ', '-': ' ', '.': ' ', '/': ' ', ':': ' ', ';': ' ', '<': ' ', '=': ' ', '>': ' ', '?': ' ', '@': ' ', '[': ' ', '\\': ' ', ']': ' ', '^': ' ', '_': ' ', '`': ' ', '{': ' ', '|': ' ', '}': ' ', '~': ' ', '«': ' ', '»': ' '}
les médiateurs confirment « l'absence de solution parfaite ». 
les médiateurs confirment   l absence de solution parfaite    


In [33]:
# stopwords
tokens = word_tokenize(new_sentence)
print(tokens)

from nltk.corpus import stopwords


# print("=== stopwords - français:")
# print(sorted(stopwords.words('french')))

list_stopwords = stopwords.words('french') + ['les', 'de']

# # equivalent:
# stopwords = stopwords.words('french')
# stopwords.append('les')
# stopwords.append('de')


# 
tokens_sans_stopwords = [w for w in tokens if (w not in list_stopwords) ]

## Equivalent 
tokens_sans_stopwords = []
for w in tokens:
    if w not in list_stopwords:
        tokens_sans_stopwords.append(w)



print(tokens_sans_stopwords)


['les', 'médiateurs', 'confirment', 'l', 'absence', 'de', 'solution', 'parfaite']
['médiateurs', 'confirment', 'absence', 'solution', 'parfaite']


In [39]:
# Enlevons la ponctuation

df['text_no_punctuation'] = df.text.apply(lambda 
    r : ( r.translate(translator) ) 
)
df.head()



,text,text_no_punctuation
0,"André Bauer, le Bonhomme de St-Dié ; Alain Dag...",André Bauer le Bonhomme de St Dié Alain Dag...
1,Le Smash Entente Club de Lunéville (SECL) a re...,Le Smash Entente Club de Lunéville SECL a re...
2,"En tout cas, du côté du PS Dole qui reste en c...",En tout cas du côté du PS Dole qui reste en c...
3,« Le nombre des donneurs était en légère baiss...,Le nombre des donneurs était en légère baiss...
4,ELOYES._ Les Ramoncenais n'auront tenu qu'une ...,ELOYES Les Ramoncenais n auront tenu qu une ...


In [40]:
# Tokenizer

df['tokens_all']  = df.text_no_punctuation.apply(
    lambda r : word_tokenize(r.lower())
)

df.head()

,text,text_no_punctuation,tokens_all
0,"André Bauer, le Bonhomme de St-Dié ; Alain Dag...",André Bauer le Bonhomme de St Dié Alain Dag...,"[andré, bauer, le, bonhomme, de, st, dié, alai..."
1,Le Smash Entente Club de Lunéville (SECL) a re...,Le Smash Entente Club de Lunéville SECL a re...,"[le, smash, entente, club, de, lunéville, secl..."
2,"En tout cas, du côté du PS Dole qui reste en c...",En tout cas du côté du PS Dole qui reste en c...,"[en, tout, cas, du, côté, du, ps, dole, qui, r..."
3,« Le nombre des donneurs était en légère baiss...,Le nombre des donneurs était en légère baiss...,"[le, nombre, des, donneurs, était, en, légère,..."
4,ELOYES._ Les Ramoncenais n'auront tenu qu'une ...,ELOYES Les Ramoncenais n auront tenu qu une ...,"[eloyes, les, ramoncenais, n, auront, tenu, qu..."


In [45]:
# remove stopwords

def remove_stopword(tokens):
     return [w for w in tokens if (w not in list_stopwords) ]

# Verifier que ca marche
remove_stopword(tokens)

# appliquer a la dataframe

df['tokens'] = df.tokens_all.apply(
    lambda tks : remove_stopword(tks) 
)

df.head()

,text,text_no_punctuation,tokens_all,tokens
0,"André Bauer, le Bonhomme de St-Dié ; Alain Dag...",André Bauer le Bonhomme de St Dié Alain Dag...,"[andré, bauer, le, bonhomme, de, st, dié, alai...","[andré, bauer, bonhomme, st, dié, alain, dagos..."
1,Le Smash Entente Club de Lunéville (SECL) a re...,Le Smash Entente Club de Lunéville SECL a re...,"[le, smash, entente, club, de, lunéville, secl...","[smash, entente, club, lunéville, secl, a, rep..."
2,"En tout cas, du côté du PS Dole qui reste en c...",En tout cas du côté du PS Dole qui reste en c...,"[en, tout, cas, du, côté, du, ps, dole, qui, r...","[tout, cas, côté, ps, dole, reste, course, acc..."
3,« Le nombre des donneurs était en légère baiss...,Le nombre des donneurs était en légère baiss...,"[le, nombre, des, donneurs, était, en, légère,...","[nombre, donneurs, légère, baisse, lors, derni..."
4,ELOYES._ Les Ramoncenais n'auront tenu qu'une ...,ELOYES Les Ramoncenais n auront tenu qu une ...,"[eloyes, les, ramoncenais, n, auront, tenu, qu...","[eloyes, ramoncenais, tenu, heure, face, réser..."


In [51]:
df.loc[5].values

array([ "Heureusement, l'affaire ne relève en rien du trafic. L'avocat de la défense fait remarquer qu'il s'agit en tout et pour tout de trente kilos de viande, d'ailleurs isolés du reste du stock, soit 6 pâtés de canard en croûtes pâtissiers, 12 taboulés et 3 mousses de canard. Juste un casse-croûte, quand la société en est à 2000 livraisons et trente tonnes vendues chaque semaine, pour un chiffre d'affaires de 30 MF annuels. « Les dates de péremption sont calculées au plus juste. Or, les produits incriminés étaient périmés d'un jour, ou d'une semaine maximum. Ils étaient encore tout à fait consommables. Et les congeler juste avant la date de péremption, c'est ce que font toutes les ménagères en gérant leur frigo ! »",
       'Heureusement  l affaire ne relève en rien du trafic  L avocat de la défense fait remarquer qu il s agit en tout et pour tout de trente kilos de viande  d ailleurs isolés du reste du stock  soit 6 pâtés de canard en croûtes pâtissiers  12 taboulés et 3 mousses de

RangeIndex(start=0, stop=1000, step=1)